<a href="https://colab.research.google.com/github/thibaud-perrin/paramete-efficient-finetuning/blob/main/notebooks/load_lora_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning and Using LoRA Adapters for OPT Models

This notebook demonstrates how to fine-tune and use LoRA (Low-Rank Adaptation) adapters with OPT language models for efficient task-specific training. By leveraging LoRA and pre-trained adapters from the Hugging Face Hub, we show how to:

1. Load a pre-trained OPT model with LoRA adapters.
2. Use efficient 8-bit training with `bitsandbytes` for faster and memory-optimized model loading.
3. Tokenize input text and generate task-specific outputs using the fine-tuned model.

This notebook is ideal for exploring lightweight fine-tuning techniques on large language models while maintaining high performance and low computational overhead.


## Libraries

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
import transformers

from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig


In [3]:
print(torch.cuda.is_available())

True


## Load adapters from the Hub

In [4]:
peft_model_id = "ybelkada/opt-6.7b-lora"
config = PeftConfig.from_pretrained(peft_model_id)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=quantization_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/332 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['bias', 'enable_lora', 'fan_in_fan_out', 'lora_alpha', 'lora_dropout', 'merge_weights', 'r', 'target_modules'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['enable_lora', 'merge_weights'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## Test Model

In [16]:
batch = tokenizer("Machine learning is:\n", return_tensors='pt').to(model.device.type)

with torch.amp.autocast(model.device.type):
  output_tokens = model.generate(
      **batch,
      max_new_tokens=100,
      do_sample=True,
      temperature=0.7,
      top_p=0.9
  )

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Machine learning is:

A subset of statistics and machine learning that uses computers to learn and improve.

The process of training a machine to perform a task or function by providing it with examples of that task or function and having it find patterns or relationships among those examples.

The act of teaching a machine to perform a task or function.

The process of teaching a machine to perform a task or function.

The process of teaching a machine to perform a task or function.

The process


In [19]:
batch = tokenizer("who was the french president in 2007?\n The french president was", return_tensors='pt').to(model.device.type)

with torch.amp.autocast(model.device.type):
  output_tokens = model.generate(
      **batch,
      max_new_tokens=100,
      do_sample=True,
      temperature=0.7,
      top_p=0.9
  )

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 who was the french president in 2007?
 The french president was Nicolas Sarkozy.
I was thinking of the guy who lost to Sarkozy in 2007.
Ah, I see. I think that was François Mitterrand.
